In [25]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import transforms, datasets
import pytorch_lightning as pl
from pytorch_lightning.metrics import Accuracy
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [26]:
class CNN_HD(pl.LightningModule):

	def __init__(self):
		super().__init__()
		self.convs = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2, bias=True),
		                           nn.ReLU(),
		                           nn.MaxPool2d(kernel_size=2),
		                           nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True),
		                           nn.ReLU(),
		                           nn.MaxPool2d(kernel_size=2, stride=2))
		self.fc = nn.Sequential(nn.Linear(16*5*5, 120),
		                        nn.ReLU(),
								nn.Linear(120, 84),
		                        nn.ReLU(),
								nn.Linear(84, 10))
		self.loss_fn = nn.CrossEntropyLoss()
		self.accuracy = 0
		self.size = 0

	def forward(self, img):
		x = self.convs(img)
		x = x.view(-1, 16*5*5)
		x = self.fc(x)
		return x

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
		return optimizer

	def training_step(self, batch, batch_idx):
		imgs, labels = batch
		predictions = self(imgs)
		loss = self.loss_fn(predictions, labels)
		result = pl.TrainResult(loss)
		result.log('train_loss', loss, prog_bar=True)
		return result

	def test_step(self, batch, batch_idx):
		imgs, labels = batch
		predictions = self(imgs)
		loss = Accuracy(predictions, labels)
		result = pl.EvalResult(loss)
		result.log('test_loss', loss)
		return result

In [27]:
train_dataset = datasets.MNIST('../data', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

test_dataset = datasets.MNIST('../data', train=False, download=False,
                        transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model = CNN_HD()
trainer = pl.Trainer(gpus=1, benchmark=True, max_epochs=5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [28]:
trainer.fit(model, train_loader)


  | Name    | Type             | Params
---------------------------------------------
0 | convs   | Sequential       | 2 K   
1 | fc      | Sequential       | 59 K  
2 | loss_fn | CrossEntropyLoss | 0     
Saving latest checkpoint..


Epoch 9: 100%|██████████| 600/600 [00:05<00:00, 102.60it/s, loss=0.011, v_num=10, train_loss=0.00219] 


1

In [29]:
result = trainer.test(model, test_loader)


Testing:  99%|█████████▉| 9933/10000 [00:10<00:00, 883.58it/s] --------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(0.0356, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 10000/10000 [00:11<00:00, 906.13it/s]
